In [3]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


import pandas as pd
import plotly.express as px




def Equ_Mar_Ratio_All(currency, date):
    df = pd.read_excel('RiskControl.xlsx')
    df = df[df['日期']==date]
    E = df[str(currency)+'_eqt'].sum()
    M = df[str(currency)+'_mar'].sum()
    return E,M,'%.f%%'%((E / M) * 100)

    if M != 0:
        return E,M,'%.f%%'%((E / M) * 100)
    else:
        return E,M,'nan%%'

#-------------------------------------------------------------------------------#
def Equ_Mar_Ratio_Each(currency, date, broker, profit_loss=0):
    df = pd.read_excel('RiskControl.xlsx')
    df = df[(df['日期']==date) & (df['上手']==broker)]
    E = df[str(currency)+'_eqt'].sum() + profit_loss
    M = df[str(currency)+'_mar'].sum()
    if M != 0:
        return E,M,'%.f%%'%((E / M) * 100)
    else:
        return E,M,'nan%%'
#-------------------------------------------------------------------------------#
def Up_3(broker):
    shs = pd.ExcelFile('RiskControl.xlsx').sheet_names
    df = pd.read_excel('RiskControl.xlsx', sheet_name=shs[-1])
    df['商品月份'] = df['商品'] + '-' + df['月份']
    df['淨口數'] = df['買方口數'] - df['賣方口數']
    df_up = df[(df['占比']>=0.03) & (df["上手"]==broker) & (df['幣別']=='USD')]
    df_up = df_up.sort_values(by='占比')

    def zero_settlement(row):
        if percent_num(row['商品月份']) :
            return  row["結算價"]
        else:
            return  row["結算價"].split('.')[0]       

    return tuple(df_up['商品月份']), tuple(df_up.apply(zero_settlement ,axis=1)), tuple(df_up['淨口數'])



#-------------------------------------------------------------------------------#
def percent_num(product):
    percent_csv = pd.read_csv('商品轉換.csv', usecols=['商品名', '小數位'],
                                 encoding='utf-8', dtype = {"小數位": "int"})
    percent_csv.drop_duplicates(inplace=True)
    percent_csv.set_index("商品名" , inplace=True)
    
    return percent_csv.loc[product.split("-")[0],'小數位']

#-------------------------------------------------------------------------------#
def profitloss_currency(product, diff, lot):
    percent_csv = pd.read_csv('商品轉換.csv', usecols=['商品名', '幣別', '1大點'],
                                 encoding='utf-8')
    percent_csv.drop_duplicates(inplace=True)
    percent_csv.set_index("商品名" , inplace=True)

    Name = product.split("-")[0]

    return int(lot * diff * percent_csv.loc[Name, '1大點']) , percent_csv.loc[Name, '幣別']



#-------------------------------------------------------------------------------#
def line_lot_use(top_num = 10): # 取前幾!!!!!!
    shs = pd.ExcelFile('RiskControl.xlsx').sheet_names
#     shs.sort(reverse=True)

    for date in shs[1:]:
        df = pd.read_excel('RiskControl.xlsx', sheet_name=date,
                          usecols=['上手','商品','月份','買方口數','賣方口數','加總'])
        df = df[df['加總']>0]
        df['總口數'] = df['買方口數'] - df['賣方口數']
        df = df.groupby(['商品'], as_index=False).sum()
        df['日期'] = date
        df = df[['商品', '總口數', '日期']]
        if shs[1] == date:
            df = df.sort_values(['總口數'],ascending=False)
            df = df[0:top_num]
            top10 = df['商品'][0:top_num]
            df_all = df.copy()
        else:
            df = df[df['商品'].isin([*top10])]
            if len(df) != top_num:
                for i in top10:
                    if i not in df['商品'].values:
                        df = df.append({'商品':i,'日期':date,'總口數':0}, ignore_index=True)
            df_all = pd.concat([df_all, df], axis=0)


    fig = px.line(df_all, x = '日期' ,y = '總口數',
                 color = '商品',log_y = True,
                 line_shape = 'spline',
                 title = '留倉口數趨勢分析',
                 labels = {'總口數':'總口數(取公司前10)'}
                 )


    fig.update_layout(yaxis = dict(tickmode = 'array',
                                    tickvals = [ 100,200, 500, 1000, 5000, 10000, 100000],
                                    ticktext = ['100','200', '500', '1000', '5000', '10000',"100000"],
                                    titlefont = dict(family = "微軟正黑體",size = 20)         
                                    ),

                      xaxis = dict(tickangle = 45,
                                     titlefont = dict(family = "微軟正黑體",size = 20 )
                                    ),
                      title_font=dict(size=25,family='微軟正黑體')
                        )
    return fig.to_html(include_plotlyjs='cdn')

#-------------------------------------------------------------------------------#

def sunburst_margin_use(margin_ratio=0.03):
    
    shs = pd.ExcelFile('RiskControl.xlsx').sheet_names
    date_ = shs[-1]  #        日期修改在這裡!!!!!!!!!

    df = pd.read_excel('RiskControl.xlsx', sheet_name=date_, 
                       usecols=['上手', '商品', '月份', "幣別", '占比', '調整後保證金'])
    df["日期"] = date_
    brokers = df['上手'].unique()
    
    for broker in brokers:
        df1 = df[df['上手'].isin([broker]) & df['幣別'].isin(['USD'])]
        df_up5 = df1[df1["占比"] >= margin_ratio]
        df_low5 = df1[df1["占比"] < margin_ratio]
        new = pd.DataFrame.from_dict({"上手":[broker],#
                                      "商品":['Other'],#
                                      "月份":["Other"],#
                                      "幣別":['USD'],
                                      "占比":[df_low5.占比.sum()],#
                                      "調整後保證金":[df_low5.調整後保證金.sum()],#
                                      "日期":[date_]})#
        df_up5 = df_up5.append(new, ignore_index=True)
        df_all_up5 = df_up5 if broker == brokers[0] else pd.concat([df_up5, df_all_up5], axis=0)

    fig = px.sunburst(df_all_up5, path=["上手", "商品","月份"],
                      values='調整後保證金',
                      width=800,
                      height=700,
                      color = '上手',
                      labels  = {"調整後保證金":"保證金"},
                      color_discrete_sequence=px.colors.qualitative.G10,
                      maxdepth = 2,
                      branchvalues='total',
                      title = date_ + " 保證金分析"
                      )

    fig.update_traces(textinfo = 'label+percent entry+value',
                      insidetextfont=dict(size=25,family='Times New Roman')
                      )
    fig.update_layout(title_font=dict(size=25,family='微軟正黑體'))
    fig.update_layout(margin=dict(t=50, l=150, r=0, b=0))

    return fig.to_html(include_plotlyjs='cdn')

#-------------------------------------------------------------------------------#

def bar_oi_use(limit_lots = 20):   #參考口數
    
    shs = pd.ExcelFile('RiskControl.xlsx').sheet_names
    date_ = shs[-1] # 日期修改在這裡!!!!
#         broker = "PHI"  # 上手輸入在這裡!!!!(暫不用)

    df = pd.read_excel('RiskControl.xlsx', sheet_name=date_,
                       usecols=['上手','商品','月份','買方口數','賣方口數','加總'])
    df["日期"] = date_
    df["商品月份"] = df["商品"] + ' - ' + df["月份"]
    df1 = df.copy()
    df1["買賣口數"] = "S"
    del df1["買方口數"]
    df1 = df1.rename(columns={"賣方口數":"口數"})
    df1 = df1[(df1["加總"]>0) & (df1["口數"]<0)]
    df1["口數"] = df1["口數"].apply(lambda x : x*-1)
    df["買賣口數"] = "B"
    del df["賣方口數"]
    df = df.rename(columns={"買方口數":"口數"})
    df = df[(df["口數"]>0) & (df["加總"]>0)]
    df = pd.concat([df, df1], axis=0)
    df = df[df["口數"]>limit_lots]
    # df = df[df["上手"]==broker]
    df = df.sort_values(by='口數')

    barchart = px.bar(data_frame=df,
                      x="商品月份",y="口數",color="買賣口數",
                      opacity=0.9,orientation="v",
                      barmode='relative',
                      text='口數',
                      hover_data=['口數','上手'],
                      title= date_ +' 留倉口數趨勢分析',
                      labels={"買賣口數":"買賣",
                             '口數':'總口數(超過20口的商品)'},
                      width=950,
                      height=700,
                      log_y = True
                      )
    barchart.update_layout(yaxis = dict(tickmode = 'array',
                                        tickvals = [0, 10, 100, 1000, 10000, 100000],
                                        ticktext = ['1', '10', '100', '1000', '10000',"100000"],
                                        titlefont = dict(family = "微軟正黑體",
                                                         size = 20,                                                     
                                                         color = "black")         
                                        ),

                            xaxis = dict(tickangle = 45,
                                         titlefont = dict(family = "微軟正黑體",
                                                          size = 20,color = "black" )
                                        ),
                           title_font=dict(size=25,family='微軟正黑體')
                            )
    barchart.update_traces(textposition = "outside")
    return barchart.to_html(include_plotlyjs='cdn')
#-------------------------------------------------------------------------------#

def pie_equity(date_):
    df = pd.read_excel('RiskControl.xlsx',usecols=['日期','上手','USD_eqt'])
    df = df[df['日期']==date_]
    pie_eqt = px.pie(df, values='USD_eqt', color='上手',names='上手',
                     color_discrete_map={"ADM":"rgb(255,217,47)","PHI":"#DC3912",
                                        "SCG":"#3366CC","IFP":"#2CA02C",},
                     template = "plotly") #plotly_dark,plotly
    pie_eqt.update_traces(textinfo = 'label+percent',
                          insidetextfont=dict(size=35,
                                              family='Times New Roman',
                                              color="white"),
                          outsidetextfont=dict(size=30,
                                               family='Times New Roman'),
                          showlegend=False)
    pie_eqt.update_layout(margin=dict(t=0, l=0, r=80, b=0),
                          title_font=dict(size=50,family='Times New Roman'))
    pie_eqt.write_image('pie_eqt.png', engine="kaleido")
